In [56]:
from rubin_scheduler.scheduler.example import example_scheduler
from rubin_scheduler.scheduler.schedulers import core_scheduler
import numpy as np
import healpy as hp
import pandas as pd
# import healsparse
from rubin_scheduler.scheduler.utils import SimTargetooServer, TargetoO
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from astropy.time import Time

from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.surveys.too_scripted_surveys import ToOScriptedSurvey

import rubin_scheduler.scheduler.basis_functions as basis_functions
import matplotlib.pylab as plt
import rubin_sim

import os
os.environ["RUBIN_SIM_DATA_DIR"] = "/project/scheduler"
os.environ["SIMS_SKYBRIGHTNESS_DATA"] = "/project/scheduler/skybrightness_pre_full"

import pickle

from rubin_scheduler.site_models import seeing_model,seeing_data
from rubin_scheduler.site_models import cloud_data,cloud_model
from rubin_scheduler.site_models import wind_data

from copy import deepcopy

%matplotlib inline

In [57]:
for k in os.environ.keys():
    if k.lower().__contains__("rubin"):
        print(k)

OBS_RUBINGENERICCAMERA_DIR
SETUP_OBS_RUBINGENERICCAMERA
RUBIN_SIM_DATA_DIR


In [58]:
import rubin_scheduler
rubin_scheduler.__version__

'3.16.0'

# Workflow

## Setup

- Read in the ToO data in the .csv's
- Use the 'standard' LSST scheduler
- Ensure that the mjd's can be updated to reflect the MJD of the injected ToO

## For each ToO

- Load the ToO data into the scheduler
- Set the scheduler start time to be -2 days before the ToO, and to end +2 days after the ToO to ensure that we have sufficient coverage
- Add the ToO to the scheduler.
- Run the ToO sim-runner for the duration specified.

## Desired outputs

- Pickle/zip file of the observation array
- gif of the observations per band
- plot of probability coverage or coverage fraction as a function of time, separated by band
- statistics of cumulative probability coverage once and twice

## Setup

In [59]:
del ToODF

In [60]:
dataDir = "/home/smacbride/too_mdc/data"
msk = [x.endswith(".csv") for x in os.listdir(dataDir)]
for f in np.array(os.listdir(dataDir))[msk]:
    if 'ToODF' in locals():
        ToODF = pd.concat((pd.read_csv(os.path.join(dataDir,f)),ToODF),axis=0,ignore_index=False)
    else:
        ToODF = pd.read_csv(os.path.join(dataDir,f))
ToODF.set_index("Unnamed: 0",inplace=True)
ToODF.index.names = ["Date generated"]
ToODF=ToODF[[not x.startswith("MS") for x in ToODF["source"]]] # Removing the hourly gw events

In [61]:
np.unique(ToODF["alert_type"],return_counts=True) # All of the alert types

(array(['GW_case_B', 'GW_case_D', 'lensed_BNS_case_A', 'neutrino'],
       dtype=object),
 array([47, 85,  8, 88]))

In [62]:
ToODF["source"][ToODF["alert_type"]=="lensed_BNS_case_A"]

Date generated
2025-10-26 02:38:17.701000+00:00     S251026i
2025-10-19 02:38:16.792000+00:00     S251019i
2025-10-24 15:30:47.958000+00:00    S251024bu
2025-10-17 15:30:47.163000+00:00    S251017bu
2025-10-03 15:30:44.096000+00:00    S251003bz
2025-10-05 02:38:16.425000+00:00     S251005i
2025-10-10 15:30:47.897000+00:00    S251010bv
2025-10-12 02:38:18.746000+00:00     S251012i
Name: source, dtype: object

#### Instantiate the example scheduler to pull the base_survey survey list, and add it to a new CoreScheduler

In [63]:
exsched = example_scheduler()

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
cat: /opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/../.git/refs/heads/main: No such file or directory
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/surveys/base_survey.py:570: FutureWarning: setting dither to bool deprecated, swapping to dither='night' (blob_long, gr)
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/surveys/base_survey.py:570: FutureWarning: setting dither to bool deprecated, swapping to dither='night' (blob_long, ri)
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/surveys/base_survey.py:570: FutureWarning: setting dither to bool deprecated, swapping to dithe

Optimizing ELAISS1
Optimizing XMM_LSS
Optimizing ECDFS
Optimizing COSMOS
Optimizing EDFS_a


/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/surveys/base_survey.py:570: FutureWarning: setting dither to bool deprecated, swapping to dither='night' (greedy)
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/surveys/base_survey.py:570: FutureWarning: setting dither to bool deprecated, swapping to dither='night' (twilight_near_sun)
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/surveys/base_survey.py:570: FutureWarning: setting dither to bool deprecated, swapping to dither='night' (twilight_near_sun)
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/surveys/base_survey.py:570: FutureWarning: setting dither to bool deprecated, swapping to dither='night' (twilight_near_sun)
  warnings.warn(

In [64]:
surveys = exsched.survey_lists # This contains all of the surveys that the usual survey has (checked by Sean)
nside = 32
camera = "LSST"
keep_rewards = True
telescope = "rubin"

In [65]:
coreSchedulerForTesting = core_scheduler.CoreScheduler(surveys=surveys,
                            nside=nside,
                            camera=camera,
                            keep_rewards=keep_rewards,
                            telescope=telescope,)
# Afterward, need to update coreSchedulerForTesting.survey_start_mjd

#### Instantiate the model observatory

In [66]:
modelObsForTesting = ModelObservatory(nside=nside, 
                                      mjd=None, 
                                      mjd_start=np.float64(60980.5), # Update this post-mortem
                                      alt_min=5.0, 
                                      lax_dome=True, 
                                      cloud_limit=0.3, 
                                      sim_to_o=None, # Update this post-mortem
                                      park_after=10.0, 
                                      init_load_length=10, 
                                      kinem_model=None, 
                                      seeing_db=None, 
                                      seeing_data=None, # Update this post mortem
                                      cloud_db=None, 
                                      cloud_offset_year=0, 
                                      cloud_data=None, # Update this post mortem
                                      downtimes=None, 
                                      no_sky=False, 
                                      wind_data=None, # Leaving as none for now, but could be updated later
                                      starting_time_key='sun_n12_setting', 
                                      ending_time_key='sun_n12_rising', 
                                      sky_alt_limits=[[20, 40], [40, 60], [20, 86]], 
                                      sky_az_limits=[0, 180], 
                                      telescope='rubin', 
                                      resolve_rotskypos=True)

In [67]:
np.unique(ToODF['alert_type'])

array(['GW_case_B', 'GW_case_D', 'lensed_BNS_case_A', 'neutrino'],
      dtype=object)

## Define many functions that we need to help with the sim-runner generation

In [70]:
def getToO_object(row,ID):
    """
    A function to create the ToO object based on the qualities of 
    the ToO data
    """
    footprint = getFootprintFromRow(row)
    center_rad = getCenterFromFootprint(footprint)
    ToO= TargetoO(
                ID,
                footprint,
                Time(row['event_trigger_timestamp']).mjd,
                getDuration(row),
                ra_rad_center=np.radians(center_rad[0]),
                dec_rad_center=np.radians(center_rad[1]),
                too_type=row['alert_type'])

    return SimTargetooServer([ToO])

def getCenterFromFootprint(footprint,nside=32):
    """
    A function to get the center of the ToO footprint (crudely)
    """
    ringOrdering = hp.pixelfunc.reorder(footprint,n2r=True)
    indices = np.argwhere(ringOrdering==1)
    middleIndex = int(np.median(indices))
    ra,dec = hp.pixelfunc.pix2ang(nside,middleIndex,nest=False)
    return ra,dec-90

def getFootprintFromRow(row):
    """
    A function to get the ToO footprint from the ToO row entry
    """
    msk = [x.__contains__("reward_map") and (not x.__contains__("nside")) for x in list(row.index)]
    myArr = row.index[msk]
    sorted_labels = myArr[np.argsort([int(s.split("reward_map")[1]) for s in myArr])]
    flattenedArr = row[sorted_labels].values
    return flattenedArr.astype(bool)
    
def getDuration(row,buffer=2):
    """
    A function that takes a row of the ToO DF as input
    Returns the appropriate duration for that ToO, with added
    time as buffer
    """
    if row["alert_type"].startswith("GW"):
        duration = 4
    elif row['alert_type']=='neutrino':
        duration = 7
    elif row['alert_type'].startswith('lensed_BNS'):
        duration = 3
    else:
        raise Exception(f"Supplied alert type is not standard. The current supported types are \
        ['GW_case_B', 'GW_case_D', 'lensed_BNS_case_A', 'neutrino']. Provided type is {row['alert_type']}. \
        Setting duration to 4 days.")
        duration=4
    return duration+buffer


def getMJDStart(row,buffer=2):
    """
    A function that takes a row of the ToO DF as input
    Returns the appropriate duration for that ToO, with added
    time as buffer
    """
    return Time(row["event_trigger_timestamp"]).mjd - buffer


def getEnvironmentData(myTime):
    """
    A wrapper function that take astropy.time.Time object as input, 
    and returns the environment model functions for that associated time
    """
    
    return getSeeingData(myTime),getCloudData(myTime)
    

def getSeeingData(myTime):
    """
    A wrapper function that take astropy.time.Time object as input, 
    and returns the seeing data for that associated time
    """
    
    return seeing_data.SeeingData(myTime)
    
def getCloudData(myTime):
    """
    A wrapper function that take astropy.time.Time object as input, 
    and returns the cloud data for that associated time
    """
    
    return cloud_data.CloudData(myTime)

def getObservatory(row,myId):
    """
    A function to update the ModelObservatory object based on the 
    time of the ToO alert activity
    """
    
    seeing_data,cloud_data = getEnvironmentData(Time(getMJDStart(row),format='mjd'))

    return ModelObservatory(nside=nside, 
                     mjd=None, 
                     mjd_start=getMJDStart(row), # Updated
                     alt_min=5.0, 
                     lax_dome=True, 
                     cloud_limit=0.3, 
                     sim_to_o=getToO_object(row,myId), # Updated
                     park_after=10.0, 
                     init_load_length=10, 
                     kinem_model=None, 
                     seeing_db="/project/scheduler/site_models/seeing.db",  # THIS IS A PLACEHOLDER
                     seeing_data=None, # Updated from above
                     cloud_db=None, 
                     cloud_offset_year=0, 
                     cloud_data=cloud_data, # Updated from above
                     downtimes=None, 
                     no_sky=False, 
                     wind_data=None, # Leaving as none for now, but could be updated later
                     starting_time_key='sun_n12_setting', 
                     ending_time_key='sun_n12_rising', 
                     # sky_alt_limits=[[20, 40], [40, 60], [20, 86]], 
                     # sky_az_limits=[0, 180], 
                     telescope='rubin', 
                     resolve_rotskypos=True)

def getScheduler(row):
    """
    A function to update the CoreScheduler object based on the 
    time of the ToO alert activity
    """

    return core_scheduler.CoreScheduler(surveys=surveys,
                            nside=nside,
                            camera=camera,
                            keep_rewards=keep_rewards,
                            telescope=telescope,
                            survey_start_mjd=getMJDStart(row))

def writeFiles(parentPath,row,obsDF,outputScheduler,outputObservatory):
    """
    A function to write the outputs of the sim-run to files
    """
    writePath = os.path.join(parentPath,row['source']) # Set the directory where things will be saved
    os.makedirs(writePath,exist_ok=True) # Make the directory
    
    # Save csv with all observations
    obsDF.to_csv(os.path.join(writePath,"observations.csv"))
    
    # Save scheduler state
    with open(os.path.join(writePath,"scheduler.pkl"),"wb") as scheduleFile:
        pickle.dump(outputScheduler,scheduleFile)
    
    # Save the observatory
    # with open(os.path.join(writePath,"observatory.pkl"),"wb") as observatoryFile:
    #     pickle.dump(outputObservatory,observatoryFile)

    return True

In [71]:
outPath = os.path.join(os.getcwd(),"data/strategyResults/preliminary")

## Run the scheduler simulation

In [ ]:
ID = 0

for index,row in ToODF.iterrows():

    # if row['source'] in os.listdir(os.path.join(outPath)):
    #     print(f"Skipping event {row['source']}")
    #     ID+=1
    #     continue # Skip this ToO alert
    
    if row["alert_type"]=='neutrino':
        row["event_trigger_timestamp"] = row["event_trigger_timestamp"][:-7]
    
    obs = getObservatory(row,ID) # need to update 
    sched = getScheduler(row)

    # Update the `modelObsForTesting` and `coreSchedulerForTesting` appropriately
    outputObservatory, outputScheduler, outputObservations = sim_runner(
            obs,
            sched,
            sim_duration=getDuration(row,buffer=4),
            filename=None,
            verbose=True,
        )
    
    obsDF = pd.DataFrame(outputObservations)
    
    result = writeFiles(outPath,row,obsDF,outputScheduler,outputObservatory) # Write the files for later use
    
    if bool(result):
        print(f"Finished simulation and save for ToO alert {row["source"]}")
        ID += 1

    del obsDF,outputScheduler,outputObservatory,obs,sched
    # Make plots in the path directory too

/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/skybrightness_pre/sky_model_pre.py:359: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flo

progress = 0.38%

/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in N

progress = 1.22%

/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in N

...

/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in N

progress = 25.02%

/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/model_observatory/model_observatory.py:681: UserWarning: No finite rotSkyPos value, using rotSkyPos_desired
  warnings.warn("No finite rotSkyPos value, using rotSkyPos_desired")
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to 

progress = 25.54%

/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in NObservationsCurrentSeason.Not updating season_map.
  warnings.warn(
/opt/lsst/software/stack/conda/envs/lsst-scipipe-11.0.0/lib/python3.12/site-packages/rubin_scheduler/scheduler/features/features.py:509: UserWarning: Time must flow forwards to track the feature in N

## Plots to now make
- Pickle/zip file of the observation array
- gif of the observations per band
- Image of the end of night band coverage summaries
- plot of probability coverage or coverage fraction as a function of time, separated by band
- statistics of cumulative probability coverage once and twice